# WikiGen Agent Workflow Testing

This notebook provides an environment to test the WikiGen agent-based story processing workflow using the new **BaseAgent architecture**. Each agent can be tested individually with different LLM models to demonstrate the flexibility and modularity of the system.

## 🏗️ BaseAgent Architecture

All WikiGen agents now inherit from `BaseAgent`, providing:
- **Default Models**: Each agent has configurable default provider/model
- **Override Flexibility**: Can override provider/model per call when needed  
- **Centralized LLM Logic**: Common error handling and validation
- **Clean API**: Simplified method signatures with optional parameters

## 📋 WikiGen Agents

1. **ArcSplitter Agent** - Analyzes story structure and determines arc boundaries
2. **WikiPlanner Agent** - Plans wiki structure and article organization  
3. **ArticleWriter Agent** - Generates actual wiki article content
4. **GeneralSummarizer Agent** - Creates summaries of various content types
5. **ChapterBacklinker Agent** - Creates bidirectional links between chapters and articles
6. **WikiGenOrchestrator** - Coordinates the complete workflow

Each agent demonstrates the BaseAgent pattern with different default models to show architectural flexibility.

## ⚙️ Setup

Make sure the Portkey Gateway is running to use the LLM Service:

```bash
docker run \
  --name portkey-gateway \
  -p 8787:8787 \
  portkeyai/gateway:latest
```

The following cells will:
- use the Portkey Gateway to test the LLM Service
- initialize the LLM Service
- import the WikiGen workflow agents
- load a test story from the `tests/resources/pokemon_amber/story` directory


In [1]:
# Cell 1: Setup and Configuration for LLM Service Testing
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import asyncio
import logging
from uuid import uuid4

# Set environment to skip database for LLM service testing
os.environ["SKIP_DATABASE"] = "true"
os.environ["PORTKEY_BASE_URL"] = "http://localhost:8787/v1"  # Default Portkey Gateway

# Add the backend/src directory to sys.path so we can import our modules
# This assumes you are running the notebook from the `backend/` directory or VS Code multi-root
notebook_dir = Path.cwd()
if (notebook_dir / 'src').is_dir() and (notebook_dir / 'pyproject.toml').is_file():
    # This means we're likely in the backend/ directory itself
    sys.path.insert(0, str(notebook_dir / 'src'))
elif (notebook_dir.parent / 'src').is_dir() and (notebook_dir.parent / 'pyproject.toml').is_file():
    # This means we're likely in the backend/notebooks/ directory
    sys.path.insert(0, str(notebook_dir.parent / 'src'))
else:
    print("Warning: Could not automatically add 'src/' to Python path. Please ensure your current directory allows imports from src/")

# Configure logging for better output
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
# Reduce noise from third-party loggers
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('uvicorn.access').setLevel(logging.WARNING)
logging.getLogger('shuscribe').setLevel(logging.INFO)

print("🧪 LLM Service Testing Notebook")
print("=" * 50)
print(f"Current working directory: {os.getcwd()}")
print(f"Database mode: {'IN-MEMORY' if os.environ.get('SKIP_DATABASE') == 'true' else 'SUPABASE'}")
print(f"Portkey Gateway: {os.environ.get('PORTKEY_BASE_URL', 'Not configured')}")
print("Autoreload enabled. Changes to .py files in src/ will be reloaded.")
print("\n💡 This notebook tests LLM service functionality without requiring database setup.")
print("   Perfect for testing direct API key usage and LLM provider integrations.")

🧪 LLM Service Testing Notebook
Current working directory: /home/jimnix/gitrepos/shuscribe/backend/notebooks
Database mode: IN-MEMORY
Portkey Gateway: http://localhost:8787/v1
Autoreload enabled. Changes to .py files in src/ will be reloaded.

💡 This notebook tests LLM service functionality without requiring database setup.
   Perfect for testing direct API key usage and LLM provider integrations.


In [2]:
# Cell 2: Import Modules (Updated for Supabase)
from src.config import settings
from src.services.llm.llm_service import LLMService
from src.api.dependencies import get_user_repository_dependency
from src.schemas.llm.models import LLMMessage, LLMResponse
from src.core.encryption import encrypt_api_key  # Import encryption function
from dotenv import dotenv_values
from src.schemas.db.user import UserCreate, UserAPIKeyCreate

print("✅ Modules imported successfully.")

# Display current settings
print("\n--- Current Settings ---")
print(f"DEBUG: {settings.DEBUG}")
print(f"ENVIRONMENT: {settings.ENVIRONMENT}")
print(f"DATABASE_BACKEND: {settings.DATABASE_BACKEND}")
print(f"PORTKEY_BASE_URL: {settings.PORTKEY_BASE_URL}")
if settings.DATABASE_BACKEND == "postgres":
    print(f"SUPABASE_URL: {settings.SUPABASE_URL}")
    print(f"SUPABASE_KEY: {'***' + settings.SUPABASE_KEY[-4:] if len(settings.SUPABASE_KEY) > 4 else 'Not set'}")
else:
    print("DATABASE_MODE: In-Memory (Supabase skipped)")
print("------------------------")

print("\n🔧 Import Summary:")
print("✅ LLMService - Ready for testing")
print("✅ Repository Factory - Automatic in-memory/Supabase switching") 
print("✅ Pydantic Models - Type-safe user schemas")
print("✅ Encryption - API key encryption/decryption")  # Updated
print("✅ Supabase Connection - Available when SKIP_DATABASE=false")

# WikiGen agent imports
from src.agents.wikigen import (
    WikiGenOrchestrator,
    ArcSplitterAgent,
    WikiPlannerAgent,
    ArticleWriterAgent,
    GeneralSummarizerAgent,
    ChapterBacklinkerAgent
)

from src.schemas.llm.models import ThinkingEffort

2025-07-02 22:43:35,117 - src.config - INFO - Pydantic Settings 'extra' mode set to: 'ignore' for environment: 'development'
✅ Modules imported successfully.

--- Current Settings ---
DEBUG: True
ENVIRONMENT: development
DATABASE_BACKEND: memory
PORTKEY_BASE_URL: http://localhost:8787/v1
DATABASE_MODE: In-Memory (Supabase skipped)
------------------------

🔧 Import Summary:
✅ LLMService - Ready for testing
✅ Repository Factory - Automatic in-memory/Supabase switching
✅ Pydantic Models - Type-safe user schemas
✅ Encryption - API key encryption/decryption
✅ Supabase Connection - Available when SKIP_DATABASE=false


In [3]:
# Cell 3: Initialize LLM Service (UPDATED)

print("🚀 Initializing LLM Service...")

# Use the dependency function to get user repository
from src.api.dependencies import get_user_repository_dependency

llm_service = LLMService(user_repository=get_user_repository_dependency())

print("✅ LLM Service initialized successfully!")
print(f"📁 Repository type: {type(llm_service.user_repository).__name__}")
print(f"🛡️  Database mode: {settings.DATABASE_BACKEND}")

assert llm_service.user_repository is not None

# Create test user and store API keys (following test patterns)
TEST_USER = await llm_service.user_repository.create_user(UserCreate(
    email="test@example.com",
    display_name="Test User"
))

# Store API keys from environment (following test service patterns)
env_values = dotenv_values()
stored_keys = 0
available_providers = []

for provider in LLMService.get_all_llm_providers():
    provider_id = provider.provider_id
    api_key = env_values.get(f"{provider_id.upper()}_API_KEY")
    
    if api_key:
        # Properly encrypt and store the API key
        encrypted_key = encrypt_api_key(api_key)
        await llm_service.user_repository.store_api_key(
            user_id=TEST_USER.id,
            api_key_data=UserAPIKeyCreate(
                provider=provider_id,
                api_key=api_key,
                provider_metadata={}
            ),
            encrypted_key=encrypted_key
        )
        stored_keys += 1
        available_providers.append(provider.display_name)
        print(f"   🔐 Stored and encrypted {provider.display_name} API key")

print(f"✅ Stored {stored_keys} encrypted API keys")

🚀 Initializing LLM Service...
✅ LLM Service initialized successfully!
📁 Repository type: MemoryUserRepository
🛡️  Database mode: memory
   🔐 Stored and encrypted OpenAI API key
   🔐 Stored and encrypted Google API key
   🔐 Stored and encrypted Anthropic API key
✅ Stored 3 encrypted API keys


In [4]:
# Cell 4: Load Test Story with Flexible Dependency Injection

print("�� Loading Test Story with Flexible Dependency Injection...")

# Import the flexible story loader with dependency injection
from src.utils.test.import_story import (
    load_pokemon_amber_with_repositories,
    create_test_story_with_repositories,
    load_story_with_repositories
)
from src.api.dependencies import (
    get_story_repository_dependency,
    get_workspace_repository_dependency, 
    get_user_repository_dependency
)
from src.schemas.db.story import FullStoryBase

# Get repositories using dependency injection (can be any implementation)
story_repo = get_story_repository_dependency()
workspace_repo = get_workspace_repository_dependency()
user_repo = get_user_repository_dependency()

print(f"�� Using repositories:")
print(f"   �� Story: {type(story_repo).__name__}")
print(f"   📁 Workspace: {type(workspace_repo).__name__}")
print(f"   👤 User: {type(user_repo).__name__}")

try:
    # Try to load Pokemon Amber story with injected repositories
    # Use the same user that has API keys from Cell 3
    story_result = await load_pokemon_amber_with_repositories(
        story_repository=story_repo,
        workspace_repository=workspace_repo,
        user_repository=user_repo
    )
    print("✅ Pokemon Amber story loaded successfully!")
    
except FileNotFoundError as e:
    print(f"⚠️  Pokemon Amber story not found: {e}")
    print("📝 Creating minimal test story instead...")
    story_result = await create_test_story_with_repositories(
        story_repository=story_repo,
        workspace_repository=workspace_repo,
        user_repository=user_repo,
        workspace_name="Test Story Workspace",
        user_email="test@example.com",
        user_display_name="Test User"
    )
    print("✅ Test story created successfully!")

# Display story summary
print()
print(story_result.summary())

# Extract variables for agent testing (clean interface)
STORY: FullStoryBase = story_result.story  # For agent compatibility
WORKSPACE_ID = story_result.workspace_id  # For repository operations
USER_ID = TEST_USER.id  # For user context
STORY_REPO = story_result.story_repository  # Individual repository access
WORKSPACE_REPO = story_result.workspace_repository
USER_REPO = story_result.user_repository

print(f"\n🎯 Ready for agent testing!")
print(f"   📖 STORY: {STORY.metadata.title} ({len(STORY.chapters)} chapters)")
print(f"   📁 WORKSPACE_ID: {WORKSPACE_ID}")
print(f"   👤 USER_ID: {USER_ID}")
print(f"   🔑 Repository types: {type(STORY_REPO).__name__}")

# Verify we're using the same user that has API keys
print(f"\n🔑 User verification:")
print(f"   �� Current user ID: {TEST_USER.id}")
print(f"   👤 Cell 3 user ID: {TEST_USER.id}")
print(f"   ✅ Same user: {TEST_USER.id == TEST_USER.id}")

# Example: Load any story directory with the same format
print(f"\n💡 Example: Load any story directory with _meta.xml format:")
print(f"   story_result = await load_story_with_repositories(")
print(f"       story_directory=Path('/path/to/your/story'),")
print(f"       story_repository=story_repo,")
print(f"       workspace_repository=workspace_repo,")
print(f"       user_repository=user_repo,")
print(f"       workspace_name='Custom Workspace',")
print(f"       user_email='test@example.com'  # Use same user as Cell 3")
print(f"   )")

�� Loading Test Story with Flexible Dependency Injection...
�� Using repositories:
   �� Story: MemoryStoryRepository
   📁 Workspace: MemoryWorkspaceRepository
   👤 User: MemoryUserRepository
✅ Pokemon Amber story loaded successfully!

�� Story: Pokemon: Ambertwo
✍️  Author: ChronicImmortality
�� Chapters: 8
�� Words: 19,378
📁 Workspace: 17c1b055-ea8a-4220-bdc7-159ac158dd7f
👤 User: 6b631c84-c6b5-497d-b4a9-a39fbf3c6bd7

🎯 Ready for agent testing!
   📖 STORY: Pokemon: Ambertwo (8 chapters)
   📁 WORKSPACE_ID: 17c1b055-ea8a-4220-bdc7-159ac158dd7f
   👤 USER_ID: 170ba109-1a06-44c8-9388-dd6c356aecd5
   🔑 Repository types: MemoryStoryRepository

🔑 User verification:
   �� Current user ID: 170ba109-1a06-44c8-9388-dd6c356aecd5
   👤 Cell 3 user ID: 170ba109-1a06-44c8-9388-dd6c356aecd5
   ✅ Same user: True

💡 Example: Load any story directory with _meta.xml format:
   story_result = await load_story_with_repositories(
       story_directory=Path('/path/to/your/story'),
       story_repository=stor

# Run individual agents

## 📋 WikiGen Agents

1. **ArcSplitter Agent** - Analyzes story structure and determines arc boundaries
2. **WikiPlanner Agent** - Plans wiki structure and article organization  
3. **ArticleWriter Agent** - Generates actual wiki article content
4. **GeneralSummarizer Agent** - Creates summaries of various content types
5. **ChapterBacklinker Agent** - Creates bidirectional links between chapters and articles
6. **WikiGenOrchestrator** - Coordinates the complete workflow

## ArcSplitter Agent - Streaming Analysis

**🔄 Single LLM Call with Real-time Streaming!**

The ArcSplitter agent now supports **streaming analysis** that provides real-time feedback while accumulating the final structured result. This approach uses **only one LLM call** for both user experience and final parsing.

- ⚡ **Real-time Feedback** - See analysis progress as it happens
- 🚀 **Single LLM Call** - No wasteful duplicate API calls  
- 📊 **Live Updates** - Stream response chunks as they're generated
- 🎯 **Smart Accumulation** - Parse final accumulated result into structured data
- 💰 **Cost Efficient** - One call gives you both streaming UX and structured output


In [5]:
# Cell 5: Test ArcSplitter Agent (Simplified)

print("🔄 Testing ArcSplitter Agent")
print("=" * 50)

# Import the agent and required models
from src.agents.wikigen import ArcSplitterAgent
from src.schemas.llm.models import ThinkingEffort

# Create the agent with simple configuration
arc_splitter = ArcSplitterAgent(
    llm_service=llm_service,
    temperature=0.7,
    max_tokens=16000,
    thinking=ThinkingEffort.LOW
)

print(f"🤖 Agent: {type(arc_splitter).__name__}")
print(f"📖 Story: {STORY.metadata.title} ({len(STORY.chapters)} chapters)")
print(f"👤 User: {TEST_USER.id}")

# Run the analysis
print("\n⚡ Starting analysis...")
try:
    # Stream the analysis
    async for chunk in arc_splitter.analyze_story_streaming(
        story=STORY,
        user_id=TEST_USER.id,
    ):
        # Show chunk type and length
        chunk_type = chunk.chunk_type.value.upper()
        content_length = len(chunk.content)
        print(f"[{chunk_type}] {content_length} chars", end=" ", flush=True)
    
    print("\n✅ Analysis completed!")
    
    # Get the final result
    final_result = arc_splitter.get_final_result()
    
    print(f"\n📊 Results:")
    print(f"   📖 Total arcs: {len(final_result.arcs)}")
    
    # Show each arc
    for i, arc in enumerate(final_result.arcs, 1):
        print(f"\n🏛️  Arc {i}: {arc.title}")
        print(f"   📄 Chapters: {arc.start_chapter}-{arc.end_chapter}")
        print(f"   📝 Summary: {arc.summary[:100]}...")
        
except Exception as e:
    print(f"\n❌ Analysis failed: {e}")
    import traceback
    traceback.print_exc()

print(f"\n🎯 ArcSplitter test completed!")

🔄 Testing ArcSplitter Agent
🤖 Agent: ArcSplitterAgent
📖 Story: Pokemon: Ambertwo (8 chapters)
👤 User: 170ba109-1a06-44c8-9388-dd6c356aecd5

⚡ Starting analysis...
2025-07-02 22:43:42,646 - src.agents.wikigen.arc_splitter - INFO - 🔄 Starting new analysis: bcf62408-28d4-4baa-bc7f-f152461064da
2025-07-02 22:43:42,646 - src.agents.wikigen.arc_splitter - INFO - 📐 Model Context Window: 1,048,576 tokens
2025-07-02 22:43:42,647 - src.agents.wikigen.arc_splitter - INFO - 📐 Chunk Limit: 1,043,576 tokens (after 5,000 overhead)
2025-07-02 22:43:42,647 - src.agents.wikigen.arc_splitter - INFO - 🔍 ArcSplitter Analysis Starting:
2025-07-02 22:43:42,648 - src.agents.wikigen.arc_splitter - INFO -    📖 Story: Pokemon: Ambertwo
2025-07-02 22:43:42,648 - src.agents.wikigen.arc_splitter - INFO -    📊 Chapters: 8, Words: 19378
2025-07-02 22:43:42,650 - src.agents.wikigen.arc_splitter - INFO -    🧮 Total tokens: 27436, Chunk limit: 1043576
2025-07-02 22:43:42,651 - src.agents.wikigen.arc_splitter - INFO -   

2025-07-02 22:43:42,717 - src.services.llm.llm_service - INFO - Set strict_open_ai_compliance=False for thinking mode
2025-07-02 22:43:42,719 - src.services.llm.llm_service - INFO - Model gemini-2.5-flash-lite-preview-06-17 supports structured output - using response_format parameter
2025-07-02 22:43:42,720 - src.services.llm.llm_service - INFO - Using simplified schema for Google Gemini (removed validation constraints)
2025-07-02 22:43:42,720 - src.services.llm.llm_service - INFO - Making LLM request: provider=google, model=gemini-2.5-flash-lite-preview-06-17, gateway=http://localhost:8787/v1, streaming=True
2025-07-02 22:43:42,720 - src.services.llm.llm_service - INFO - Calculated thinking budget: 3686 tokens for low effort on google/gemini-2.5-flash-lite-preview-06-17
2025-07-02 22:43:42,721 - src.services.llm.llm_service - INFO - Using thinking with 3686 budget tokens for Google model gemini-2.5-flash-lite-preview-06-17
[THINKING] 398 chars [THINKING] 508 chars [THINKING] 498 chars

In [6]:
print(final_result.model_dump_json(indent=2))

{
  "story_prediction": "The narrative is poised to explore the protagonist's journey as a trainer, their quest for identity, and their entanglement with the darker elements of the Pokemon world, particularly Team Rocket and the implications of Mewtwo's existence. Future developments will likely involve the protagonist mastering Ditto's unique abilities, acquiring new Pokemon, challenging Gyms, and uncovering the secrets of Dr. Fuji's research and his connection to Team Rocket. The mystery surrounding Mewtwo's escape and its impact on the world will likely drive significant plot points. Interactions with characters like Erika suggest a focus on uncovering local conspiracies or challenges within Celadon City, which will serve as a stepping stone for broader adventures. The protagonist's meta-knowledge from their previous life will continue to be a source of both advantage and potential danger, especially if it draws unwanted attention from powerful organizations. The ethical implication

## WikiPlanner Agent

In [ ]:
#
#
#

## ArticleWriterAgent

In [ ]:
#
#
#

## GeneralSummarizerAgent

In [ ]:
#
#
#

## WikiGenOrchestrator

In [ ]:
#
#
#